In [1]:

import pandas as pd

from models import evaluate_model, stan_init,\
load_prophet_model, load_scaler\
,get_model_metrics, model_add_observations, get_prophet_predictions, X_train_add_observations, get_y_test_new

from plots import plot_forecast_measure
from utils import get_data
from preprocessing import select_df_columns, df_interpolate_and_dropna

from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
sc = load_scaler('models/20210605153739_prophet_std_scaler.bin')

In [3]:
model = load_prophet_model('models/20210605153739_prophet1.json')

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [4]:
%%time
df , df_raw = get_data('./data')

Wall time: 29.4 s


In [5]:
%%time
df_prep = df.pipe(select_df_columns, ["ET_rete (potenza_termica_oraria)"])\
    .pipe(df_interpolate_and_dropna)

Wall time: 15 ms


In [6]:
df_test = df_prep.loc["2020":]

In [7]:
df_train = df_prep.loc[:"2019"]

In [10]:
%%time
window=48
step_forward = 24
d = {}
for i in range(1, int(df_test.shape[0]/window)+2):
    print(i)
    if i==1:
        end_window = i*window
        y_true = df_test["ET_rete (potenza_termica_oraria)"].iloc[:end_window].values
        y_pred = get_prophet_predictions(model, sc, periods=window)
        mae, mape = get_model_metrics(y_true, y_pred)
        d[i] = {"mae": mae, "mape": mape}
        print(mae, mape)
    elif i==2:
        end_window+=step_forward
        X_train_new = X_train_add_observations(df_train, df_test[["ET_rete (potenza_termica_oraria)"]].iloc[:end_window], scaler=sc)
        new_model = model_add_observations(model, X_train_new)
        y_pred_new = get_prophet_predictions(new_model, sc, periods=window)
        y_true_new = get_y_test_new(X_train_new, df_test, periods=window)
        mae, mape = get_model_metrics(y_true_new, y_pred_new)
        d[i] = {"mae": mae, "mape": mape}
        print(mae, mape)
    else:
        end_window+=step_forward
        X_train_new = X_train_add_observations(df_train, df_test[["ET_rete (potenza_termica_oraria)"]].iloc[:end_window], scaler=sc)
        new_model = model_add_observations(new_model, X_train_new)
        y_pred_new = get_prophet_predictions(new_model, sc, periods=window)
        y_true_new = get_y_test_new(X_train_new, df_test, periods=window)
        mae, mape = get_model_metrics(y_true_new, y_pred_new)
        d[i] = {"mae": mae, "mape": mape}
        print(mae, mape)

1
39.92921829623598 14.448378505782303
2
100.44632002690425 157.62769644511567
3
189.19219846804467 364.3171262073223
4
188.99676375201486 370.66962539574575
5
161.70362991570497 231.98434801346278
6
125.01809332004298 120.0920500677862
7
88.70157667327044 64.66055303275084
8
52.24343925688754 31.373693510642987
9
27.148394292922585 14.756225026641124
10
66.32638752825109 21.081639040362038
11
109.26174709848085 30.605733070892477
12
96.50896451893989 29.77519770478883
13
80.0106120688642 28.60123571993215
14
71.85334306078778 27.5326638584024
15
62.40528631344055 27.013338662186044
16
50.87905258895347 26.21650503930762
17
56.53447477118215 25.57235914845521
18
75.0664725026834 26.685774791066745
19
79.25282381449531 26.712900147763914
20
71.24668606140035 24.990165109639616
21
67.00144688587856 24.474702543804693
22
57.501612316395665 24.785576405900674
23
36.40048012848394 22.137584095452976
24
31.63150763816218 19.900269580211365
25
40.31088115433209 21.430526469810314
26
39.132036

KeyboardInterrupt: 

In [9]:
pd.DataFrame().from_dict(d, orient="index").mean()

mae     74.791522
mape    63.184480
dtype: float64